################  IMPORTING THE REQUIRED LIBRARIES

In [2]:
import numpy as np
import soundfile
import os, glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import librosa
from sklearn import preprocessing 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

############## EMOTIONS INCLUDED IN THE DATASET

In [3]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

############## EXTRACTING FEATURES FROM THE AUDIO SIGNAL USING LIBROSA

In [4]:
def extract_feature(file_name, mfcc, chroma,spectral_centroid,spectral_bandwidth,
                    spectral_rolloff,spectral_contrast,rms,spectral_flatness):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
            
            
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))             
           
            
        if spectral_centroid:
            spectral_centroid=np.mean(librosa.feature.spectral_centroid(y=X, sr=sample_rate))
            result=np.hstack((result, spectral_centroid)) 
        
        if spectral_bandwidth:
           spectral_bandwidth=np.mean(librosa.feature.spectral_bandwidth(y=X, sr=sample_rate).T)
#           print(spectral_bandwidth)
           result=np.hstack((result, spectral_bandwidth)) 
           
        if spectral_rolloff:
           spectral_rolloff=np.mean(librosa.feature.spectral_rolloff(y=X, sr=sample_rate).T)
#           print(spectral_rolloff)
           result=np.hstack((result, spectral_rolloff))
        
        if spectral_contrast:
           spectral_contrast=np.mean(librosa.feature.spectral_contrast(y=X, sr=sample_rate))
           result=np.hstack((result, spectral_contrast))
           
        if rms:
           rms=np.mean(librosa.feature.rms(y=X).T,axis=0)
           result=np.hstack((result, rms))
           
        if spectral_flatness:
           spectral_flatness=np.mean(librosa.feature.spectral_flatness(y=X))
           result=np.hstack((result, spectral_flatness))
        
        return result

##############     LOADING THE DATASET AND  EXTRACTING ALL THE FEATURES FROM IT  

In [5]:
def load_data():
    x,y=[],[]
    for file in glob.glob(r"F:\speech_project\speech-emotion-recognition-ravdess-data\Actor_*\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        feature=extract_feature(file, mfcc=True, chroma=True,spectral_centroid=True,spectral_bandwidth=True,
                                spectral_rolloff=True,spectral_contrast=True,rms=True,spectral_flatness=True)
        x.append(feature)
        y.append(emotion)
    return x,y


##############     SHUFFLING OF OBS AND RESETTING THE OBS INDEXES

In [6]:
X,y=load_data()
X=pd.DataFrame(X)
y=pd.DataFrame(y)
data=pd.concat([X,y],axis="columns")
data=data.sample(frac=1).reset_index(drop=True)
X=data.iloc[:,0:58]
y=data.iloc[:,58]

##############     LABEL ENCODING THE RESPONSE VARIABLES

In [7]:
le = preprocessing.LabelEncoder()
y=le.fit_transform(y)

##############     SPLITTING THE DATA INTO TRAIN(95%) AND TEST(5%)

In [8]:
x_train,x_test,y_train,y_test=train_test_split(X, y, test_size=0.05, random_state=42)
print((x_train.shape[0], x_test.shape[0]))

print(f'Features extracted: {x_train.shape[1]}')

(1345, 71)
Features extracted: 58


In [9]:
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)

################### HYPER-PARAMETER TUNING

In [10]:
param_mlp={'learning_rate_init':np.linspace(0.01,0.3,20),
          'hidden_layer_sizes':[(124,123,132),(210,230,260)],
          'activation':['logistic','tanh','relu'],
          'learning_rate':['constant', 'invscaling', 'adaptive']}

#################### TRAINING THE MODEL

In [11]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(random_state=2019,max_iter=500)
mlpGrid = GridSearchCV(model, param_grid=param_mlp, cv=kfold,scoring='accuracy', verbose=-3, n_jobs=-1)

# Train the model
mlpGrid.fit(x_train,y_train)

# Predict for the test set
y_pred_1=mlpGrid.predict(x_test)

print(accuracy_score(y_test, y_pred_1))
## Best Parameters
print(mlpGrid.best_params_)
print(mlpGrid.best_score_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed: 11.9min finished


0.28169014084507044
{'activation': 'relu', 'hidden_layer_sizes': (210, 230, 260), 'learning_rate': 'constant', 'learning_rate_init': 0.01}
0.308550185873606


############# SAVING THE MODEL

In [12]:
import pickle
pickle.dump(mlpGrid, open("C:\\Users\\dbda\\Desktop\\project\\MLP.pkl", 'wb'))
print("Model Saved!!")

Model Saved!!


#################### Prediction on unseen data

In [13]:

def load_test_data():
    x_test,y_test=[],[]
    
    file="F:\\speech_project\\UnSeenSet\\03-01-07-01-01-02-15.wav"
    emotion=emotions[file.split("-")[2]]
    feature=extract_feature(file, mfcc=True, chroma=True,spectral_centroid=True,spectral_bandwidth=True,
                            spectral_rolloff=True,spectral_contrast=True,rms=True,spectral_flatness=True)
    
    x_test.append(feature)
    y_test.append(emotion)
    print(x_test)
    return x_test,y_test

In [14]:
X_test_data,y_test_data=load_test_data()
print(y_test_data)

[array([-5.76542969e+02,  6.80835495e+01, -2.08600235e+01,  1.93035564e+01,
        1.35577583e+00, -1.16599786e+00, -6.15480709e+00, -5.99488688e+00,
        1.69360983e+00,  2.48043895e+00, -1.89693403e+00, -8.06386828e-01,
        1.91894567e+00,  6.07720947e+00, -9.03752983e-01, -9.56951082e-01,
        7.98137605e-01, -4.33641291e+00,  4.76005189e-02,  1.36326599e+00,
       -2.66334414e-01,  9.42056358e-01,  1.03757173e-01,  3.81496608e-01,
        4.33735341e-01,  1.11797643e+00, -5.18218040e-01,  2.82531047e+00,
        9.52483118e-02,  1.69378901e+00, -2.51021296e-01,  2.14043283e+00,
        3.39507866e+00,  3.61230636e+00,  2.16703892e+00,  2.10190296e+00,
        3.72034073e-01,  1.80648112e+00, -3.42183448e-02, -1.00127256e+00,
        5.94972134e-01,  5.97770333e-01,  5.77731550e-01,  6.29865050e-01,
        7.22877383e-01,  7.31866360e-01,  6.78925991e-01,  7.01202929e-01,
        7.34922707e-01,  7.51407385e-01,  7.65149891e-01,  6.68664038e-01,
        2.17363899e+03, 

########### LOADING SAVED MODEL

In [15]:

import pickle
loaded_model = pickle.load(open("C:\\Users\\dbda\\Desktop\\project\\MLP.pkl", 'rb'))
result = loaded_model.predict(X_test_data)
print(result)

[7]


In [16]:
print(le.inverse_transform(result))

['surprised']
